# Rapport - Classification d’images  
Comparaison de CNN et d’une méthode classique

## 1. Introduction

Ce projet vise à comparer plusieurs approches de classification d’images sur différents jeux de données, en étudiant leurs performances, leur complexité et leur comportement face à la variabilité des données.

---

## 2. Objectif global

L’objectif principal de ce projet est de comparer trois approches de classification d’images :

- Un CNN à 2 couches de convolution
- Un CNN à 3 couches de convolution
- Une méthode classique basée sur ORB + Bag of Visual Words (BoVW) + SVM

Ces méthodes ont été évaluées sur trois jeux de données distincts, afin d’analyser leur capacité de généralisation et leur robustesse selon la complexité des images.

### Choix des architectures CNN

Le CNN à 2 couches de convolution a été choisi comme modèle de référence (baseline). Cette architecture simple permet :
- D’établir un premier niveau de performance
- De limiter le temps d’entraînement
- De mieux comprendre l’impact des couches de convolution sur l’extraction de caractéristiques

Le CNN à 3 couches de convolution constitue une extension naturelle de cette baseline. L’ajout d’une couche supplémentaire permet :
- D’extraire des caractéristiques plus abstraites et hiérarchisées
- D’évaluer l’impact de la profondeur du réseau sur les performances
- De comparer le compromis entre complexité du modèle et gain de précision

Ce choix progressif (2 puis 3 convolutions) permet une comparaison équitable, en modifiant uniquement la profondeur du réseau tout en conservant une structure globale similaire.


### Choix de la méthode ORB + BoVW + SVM

En complément des CNN, une méthode classique basée sur ORB, Bag of Visual Words et SVM a été implémentée afin de disposer d’un point de comparaison ne reposant pas sur l’apprentissage profond.

Ce choix est motivé par :
- La popularité historique de cette approche en vision par ordinateur
- Son faible coût computationnel comparé aux CNN
- Sa capacité à fournir des résultats corrects sur des datasets simples

Cette méthode permet également d’évaluer l’intérêt réel des CNN face à une approche plus traditionnelle, notamment lorsque les ressources de calcul sont limitées.

 ---

## 3. Jeux de données

Trois datasets ont été utilisés tout au long du projet. Ils diffèrent par :

- Le nombre de classes
- La diversité visuelle
- La taille et la résolution des images
- La difficulté globale de la tâche de classification

Chaque dataset a été séparé en ensembles d’entraînement et de validation afin d’évaluer les performances des modèles de manière cohérente.

Dire les propriété de chaque base et expliquer nos choix : CNN 2 CNN 3 Transfert Learning Image Net et oRB. 



---

## 4. Comparaison des résultats par dataset (CNN 2 conv vs CNN 3 conv vs ORB+BoVW+SVM)

### MNIST (10 classes)

- CNN 2 convolutions
  - Meilleure accuracy validation : 99,36 %
  - Early stopping : déclenché à l’epoch 19, meilleur modèle restauré à l’epoch 14 

- CNN 3 convolutions
  - Meilleure accuracy validation : 98,95 %
  - Early stopping : déclenché à l’epoch 17, meilleur modèle restauré à l’epoch 12

- ORB + BoVW + SVM
  - Accuracy test : 63,73 % 

#### Analyse :
Sur MNIST, les deux CNN atteignent des performances très élevées (proche de 99%), ce qui montre qu’une architecture peu profonde suffit souvent sur un dataset simple et très structuré. Ici, le modèle CNN 2 convolutions fait même légèrement mieux que le 3 convolutions, ce qui suggère que l’ajout de profondeur n’apporte pas forcément un gain dans ce contexte (et peut légèrement compliquer l’optimisation).  
En comparaison, ORB+BoVW+SVM reste nettement en dessous, car la méthode repose sur des descripteurs locaux et capture moins bien la forme globale des chiffres.

### Dogs vs Cats (classification binaire)

- CNN 2 convolutions
  - Meilleure accuracy validation : 78,2 %
  - Perte de validation minimale : 0,456
  - Early stopping : déclenché à l’epoch 13, meilleur modèle restauré à l’epoch 10

- CNN 3 convolutions
  - Meilleure accuracy validation : 80,2 %
  - Perte de validation minimale : 0,434
  - Early stopping : déclenché à l’epoch 17, meilleur modèle restauré à l’epoch 14

- ORB + BoVW + SVM
  - Accuracy test : 69,3 %

#### Analyse :
Sur le dataset Dogs vs Cats, l’ajout d’une troisième couche de convolution permet une amélioration notable des performances, avec un gain d’environ 2 pourcent en accuracy sur l’ensemble de validation et une perte plus faible. Cela indique une meilleure capacité du modèle à extraire des caractéristiques discriminantes telles que les textures, formes et motifs globaux des animaux.

Le CNN à 2 convolutions offre néanmoins un compromis intéressant entre performances et temps d’entraînement. À l’inverse, la méthode ORB + BoVW + SVM montre des performances plus limitées. Bien qu’elle reste au-dessus du hasard, elle peine à capturer la variabilité visuelle importante des images naturelles, ce qui explique l’écart observé avec les CNN.

### Intel Image Classification (6 classes)

- CNN 2 convolutions
  - Meilleure accuracy validation : 82,1 %
  - Perte de validation : 0,51

- CNN 3 convolutions
  - Meilleure accuracy validation : 84,8 %
  - Perte de validation : 0,45

- ORB + BoVW + SVM
  - Accuracy test : 48,8 %

#### Analyse :
Sur le dataset Intel Image Classification, qui constitue le cas le plus complexe du projet, l’impact de la profondeur du réseau est particulièrement visible. Le CNN à 3 convolutions surpasse clairement le modèle à 2 convolutions, avec une meilleure précision et une perte de validation plus faible, traduisant une meilleure généralisation.

La méthode ORB + BoVW + SVM atteint une accuracy d’environ 49 %, ce qui reste supérieur au hasard (16,7 % pour 6 classes), mais largement inférieur aux CNN. Cette différence s’explique par la difficulté de représenter la structure globale des scènes à l’aide de descripteurs locaux uniquement, notamment lorsque plusieurs classes présentent de fortes similarités visuelles.

Ces résultats ont conduit à explorer plusieurs pistes d’optimisation, présentées dans la section suivante.

---

## 5. Nettoyage et préparation des données

Avant l’entraînement des différents modèles, une phase de nettoyage et de préparation des jeux de données a été réalisée. Cette étape est essentielle afin de garantir la qualité des données d’entrée et d’éviter des erreurs ou biais lors de l’apprentissage.

### Gestion des images corrompues ou illisibles

Lors de l’exploration des datasets, en particulier Dogs vs Cats et Intel Image Classification, certaines images se sont révélées corrompues, partiellement téléchargées ou impossibles à charger correctement par les bibliothèques de traitement d’images.

Ces images ont été détectées lors de la phase de chargement des données et supprimées du jeu de données, afin d’éviter :
- Des erreurs lors de l’entraînement
- Des interruptions de l’exécution des scripts
- Une dégradation de la qualité de l’apprentissage

### Normalisation et redimensionnement des images

Les images des datasets Dogs vs Cats et Intel présentent des tailles et des formats variés. Afin de garantir une entrée cohérente pour les réseaux de neurones convolutifs, toutes les images ont été :

- Redimensionnées à une taille fixe
- Converties dans un format compatible
- Normalisées (valeurs de pixels ramenées dans un intervalle standard)

Cette étape permet d’assurer une convergence plus stable des modèles et une comparaison équitable entre les différentes architectures.

### Équilibrage et séparation des données

Les jeux de données ont été divisés en ensembles d’entraînement et de validation de manière cohérente. Une attention particulière a été portée à conserver une répartition équilibrée des classes afin d’éviter un biais en faveur d’une classe dominante, en particulier pour la classification binaire Dogs vs Cats.

### Cas particulier du dataset MNIST

Le dataset MNIST étant déjà normalisé, équilibré et largement utilisé comme référence académique, aucune étape de nettoyage supplémentaire n’a été nécessaire. Seules des opérations de normalisation et de séparation entraînement/validation ont été appliquées.


En résumé, cette phase de nettoyage et de préparation des données a permis de garantir des données fiables et cohérentes, condition indispensable à l’obtention de résultats reproductibles et à une comparaison pertinente des différentes méthodes de classification.

---

## 6. Essais explorés mais non retenus

Au cours du projet, plusieurs pistes d’optimisation ont été explorées afin d’améliorer les performances des modèles. Toutefois, certaines d’entre elles n’ont pas été conservées dans la version finale, soit en raison de gains de performance limités, soit à cause de contraintes computationnelles importantes. Dans plusieurs cas, l’augmentation du temps d’entraînement et de la complexité ne justifiait pas les améliorations obtenues.


### Variation de la taille des batchs

Différentes tailles de batch ont été testées lors de l’entraînement des CNN. Des batchs plus petits ont parfois permis une meilleure stabilité de l’apprentissage, mais ont entraîné une augmentation significative du temps d’entraînement, rendant les expérimentations beaucoup plus longues à exécuter.  

À l’inverse, des batchs plus grands ont accéléré certaines phases d’apprentissage, mais ont souvent conduit à une convergence moins stable et à des performances de validation légèrement inférieures.

Dans l’ensemble, ces variations n’ont pas apporté d’amélioration significative et reproductible des résultats, alors qu’elles pouvaient fortement ralentir l’entraînement. Une taille de batch intermédiaire a donc été retenue, offrant un compromis satisfaisant entre stabilité, performances et temps de calcul.


### Ajustement des hyperparamètres avancés

Plusieurs ajustements supplémentaires ont également été testés, notamment :
- Des taux de dropout plus élevés
- Des variations du learning rate
- L’utilisation de régularisations plus agressives

Ces modifications ont généralement conduit à des temps d’entraînement plus longs et parfois à une instabilité de l’apprentissage, sans apporter de gain de performance clair et durable sur l’ensemble des datasets. Dans certains cas, les améliorations observées étaient marginales et ne compensaient pas la complexité supplémentaire introduite.

Nous avons donc décidé de ne pas garder ces ajustements dans la version finale.

---

## 7. Stratégies d’entraînement et de régularisation des CNN

Afin d’assurer un entraînement stable et une comparaison équitable entre les différentes architectures CNN, plusieurs mécanismes communs ont été mis en place pour l’ensemble des expériences.

### Early stopping

Le mécanisme d’early stopping a été utilisé afin de limiter le surapprentissage. L’entraînement est interrompu automatiquement lorsque la performance sur l’ensemble de validation cesse de s’améliorer pendant un certain nombre d’époques consécutives (patience). Le modèle conserve alors les poids correspondant à la meilleure performance observée.

Ce choix permet :
- D’éviter un surentraînement inutile
- De réduire le temps de calcul
- De sélectionner un modèle généralisant mieux

### Checkpointing des modèles

Un système de checkpoints a été mis en place afin de sauvegarder les poids du modèle à chaque epoch (ou lorsque la métrique de validation s’améliore). Cette stratégie permet :
- De conserver une trace des différentes étapes d’apprentissage
- De restaurer le meilleur modèle après l’early stopping
- D’analyser a posteriori l’évolution des performances

### Choix du nombre d’époques

Le nombre maximal d’époques a été fixé à 20 pour l’ensemble des CNN. Cette valeur constitue un compromis entre :
- Une durée d’entraînement raisonnable
- Une convergence suffisante des modèles

Dans la majorité des cas, l’early stopping est déclenché avant d’atteindre ce nombre maximal, ce qui confirme que cette limite est suffisante pour permettre au modèle de converger.

### Suivi des performances par epoch

À chaque epoch, les métriques de loss et accuracy ont été enregistrées sur les ensembles d’entraînement et de validation. Ce suivi permet :
- D’analyser la dynamique d’apprentissage
- De détecter d’éventuels phénomènes de surapprentissage
- De comparer finement les comportements des CNN à 2 et 3 couches de convolution

---

## 8. Problèmes rencontrés et solutions

Au cours du projet, plusieurs difficultés ont été rencontrées lors de la mise en œuvre et de l’entraînement des différents modèles de classification d’images. Cette section présente les principaux problèmes identifiés ainsi que les solutions apportées afin d’assurer la stabilité des expériences et la fiabilité des résultats.

### Surapprentissage des modèles CNN

L’un des principaux problèmes observés concerne le surapprentissage, en particulier pour les architectures les plus complexes et sur les datasets de taille limitée. Ce phénomène se manifestait par une amélioration continue des performances sur l’ensemble d’entraînement, tandis que l’accuracy de validation stagnait ou diminuait.

Solution apportée :
- Mise en place d’un mécanisme d’early stopping basé sur la performance de validation
- Restauration automatique des poids correspondant à la meilleure epoch
- Utilisation de dropout afin de limiter la dépendance excessive à certaines caractéristiques

Ces solutions ont permis d’améliorer la capacité de généralisation des modèles et de stabiliser l’apprentissage.

### Temps d’entraînement élevés

L’entraînement des CNN, en particulier avec des architectures plus profondes ou des paramètres plus coûteux, a entraîné des temps de calcul importants. Ce problème était accentué lors des tests d’optimisation (variation des batchs, des hyperparamètres ou de la résolution des images).

Solution apportée :
- Limitation du nombre maximal d’époques
- Utilisation systématique de l’early stopping pour interrompre les entraînements non prometteurs
- Abandon des configurations offrant des gains marginaux au regard du temps de calcul supplémentaire

Ces choix ont permis de conserver des temps d’entraînement raisonnables sans dégrader les performances finales.


### Sensibilité aux hyperparamètres

Les performances des modèles CNN se sont révélées sensibles à certains hyperparamètres, notamment le learning rate, la taille des batchs et le taux de dropout. Des valeurs inadaptées pouvaient conduire à une convergence lente ou instable.

Solution apportée :
- Tests progressifs et contrôlés des hyperparamètres
- Conservation de configurations simples et robustes
- Évitement des réglages trop agressifs susceptibles de dégrader la stabilité de l’apprentissage

Cette approche a permis d’obtenir des résultats reproductibles et cohérents sur l’ensemble des datasets.
